# Logging Tracing Concept for Email Workflow

As the production system is not ready yet, and even if it would be there is not enough time to gather enough test data to validate the concept a demo is needed.

To properly validate the concept the demo has to check below requirements:
* simulate every component existing in future production system
* simulate errors / full failures in specific components to validate concept
* produce large amount of realistic test data in a short amount of time

In [1]:
import datetime
import logging
from logging import config
import sys

In [2]:
case_format = "CASE::%(levelname)s::<<<%(message)s>>>"
debug_format = "DEBUG::%(asctime)s::%(levelname)s::<<<%(message)s>>>"
debug_date_format = "%Y-%m-%d %H:%M:%S"

def setup_logger(name):
    level = logging.INFO
    config.dictConfig({
        'version': 1,
        'disable_existing_loggers': True,
        'formatters': {
            'debug': {'format': debug_format, 'datefmt': debug_date_format},
            'case': {'format': case_format}
        },
        'handlers': {
            'debug_console': {
                'class': 'logging.StreamHandler',
                'level': level,
                'formatter': 'debug',
                'stream': 'ext://sys.stdout'
            },
            'case_console': {
                'class': 'logging.StreamHandler',
                'level': level,
                'formatter': 'case',
                'stream': 'ext://sys.stdout'
            },
            'debug_file': {
                'class': 'logging.FileHandler',
                'level': level,
                'formatter': 'debug',
                'filename': 'debug.log'
            },
            'case_file': {
                'class': 'logging.FileHandler',
                'level': level,
                'formatter': 'case',
                'filename': 'case.log'
            },
        },
        'loggers': {
            'debug': {
                'level': level,
                'handlers': ['debug_console', 'debug_file']
            },
            'case': {
                'level': level,
                'handlers': ['case_console', 'case_file']
            }
        }
    })
    return logging.getLogger(name)

debugLogger = setup_logger("debug")
caseLogger = setup_logger("case")

In [3]:
debugLogger.info("HELLO")
caseLogger.info("HELLOOOOOO")

DEBUG::2021-01-04 14:13:26::INFO::<<<HELLO>>>
CASE::INFO::<<<HELLOOOOOO>>>


In [4]:
class CaseCapsule:
    def __init__(self, id: str, logger: logging.Logger):
        # TODO
        self._id = id;
        self._last_seen = datetime.datetime.today()
        self._counter = 1
        self._logger = logger

    @property
    def id(self):
        return self._id

    @property
    def last_seen(self):
        return self._last_seen

    @property
    def counter(self):
        return self._counter

    def add_seconds(self, seconds):
        self._counter += 1
        timedelta = datetime.timedelta(seconds=seconds)
        self._last_seen += timedelta

    @property
    def logger(self):
        return self._logger

    def __str__(self):
        return f"Case id[{self._id}] last_seen[{self._last_seen}] counter[{self._counter}]"

In [5]:
case = CaseCapsule("123ab")
print(case)
case.add_seconds(129)
print(case)

TypeError: __init__() missing 1 required positional argument: 'logger'

In [ ]:
class Component:
    def __init__(self, pre_sub_function, post_sub_function, debug_logger, sub_component=None):
        # TODO
        self._pre_sub_function = pre_sub_function
        self._post_sub_function = post_sub_function
        self._sub_component = sub_component
        self._debug_logger = debug_logger

    def exec(self, case: CaseCapsule):
        self._debug_logger.log.debug('before calling pre sub functions')
        self._pre_sub_function(case, self._debug_logger)
        self._debug_logger.log.debug('after calling pre sub functions')
        if self._sub_component:
            self._debug_logger.log.debug('before calling sub component')
            self._sub_component.exec()
            self._debug_logger.log.debug('after calling sub component')
        self._debug_logger.log.debug('before calling post sub functions')
        self._post_sub_function(case, self._debug_logger)
        self._debug_logger.log.debug('after calling post sub functions')

In [ ]:
def test_pre_fun(case:Case, logger: logging.Logger):
    logger.log.debug('some demo pre function')
    case.add_seconds(2)